In [ ]:
# !pip install -i https://pypi.tuna.tsinghua.edu.cn/simple -U --user transformers==4.45.0 trl==0.9.6
# https://blog.csdn.net/flyfish1986/article/details/142382893

In [31]:
import warnings; warnings.filterwarnings("ignore")
import os
import sys
import json
import torch as th
# import torch_npu as th_npu
import transformers
import trl

from pprint import pp
from tqdm import trange
from transformers import DataCollatorForLanguageModeling
from trl import (SFTConfig, SFTTrainer)

In [25]:
device = th.device("cuda" if th.cuda.is_available() else "cpu")
# device = th.device("npu" if th.npu.is_available() else "cpu")
devive_cnt = th.cuda.device_count()
# devive_cnt = th.npu.device_count()
print(f"device = {device}; devive_cnt = {devive_cnt}")
print(f"torch version = {th.__version__}")
print(f"cuda version = {th.version.cuda}")
print(f"transformers version = {transformers.__version__}")
print(f"trl version = {trl.__version__}")

device = cuda; devive_cnt = 1
torch version = 2.5.1+cu121
cuda version = 12.1
transformers version = 4.45.0
trl version = 0.9.6


In [30]:
path_project = "C:/my_project/MyGit/Machine-Learning-Column"
path_data = os.path.join(path_project, "data")
path_model = "F:/LLM"
path_output = os.path.join(path_model, "output")

In [ ]:
list_sharegpt_samples = []

In [ ]:
dict_conversations = {
    "conversations": [
        # user prompt
        {
            "from": "human",
            "value": "我想知道明天济南的天气。"
        },
        # function call
        {
            "from": "function_call",
            "value": "{\"name\": \"get_weather_forecast\", \"arguments\": {\"location\": \"济南\", \"date\": \"明天\"}}"
        },
        # function response
        {
            "from": "observation",
            "value": "{\"temperature_high\": 25, \"temperature_low\": 18, \"condition\": \"多云\", \"humidity\": 60}"
        },
        # model response
        {
            "from": "gpt",
            "value": "明天济南的天气预报显示是多云，最高温度约为25摄氏度，最低温度约为18摄氏度，湿度为60%。"
        }
    ],
    # system prompt (optional)
    "system": "你是一位智能助手，能够帮助用户调用工具查询各种信息。",
    # tools schema (optional)
    "tools": "[{\"name\": \"get_weather_forecast\", \"description\": \"获取指定地点的天气预报\", \"parameters\": {\"type\": \"object\", \"properties\": {\"location\": {\"type\": \"string\", \"description\": \"要查询天气的地点\"}, \"date\": {\"type\": \"string\", \"description\": \"查询的日期\"}}, \"required\": [\"location\", \"date\"]}}]"
}


In [ ]:
for i in trange(len(df), desc="将数据集改写为 ShareGPT 格式"):
    dict_conversations = {...}
    list_sharegpt_samples.append(dict_conversations)

In [ ]:
with open(os.path.join(path_data, "sharegpt_samples.jsonl"), mode="w", encoding="utf-8") as f:
    for line in list_sharegpt_samples:
        f.write(json.dumps(line, ensure_ascii=False) + "\n")

In [19]:
d = {
    "name": "get_weather_forecast",
    "arguments": {
        "location": "济南",
        "date": "明天"
    }
}

In [ ]:
d = {
    "name": "get_weather_forecast",
    "description": "获取指定地点的天气预报",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "要查询天气的地点"
            },
            "date": {
                "type": "string",
                "description": "查询的日期"
            }
        },
        "required": ["location", "date"]
    }
}

In [22]:
json.dumps(d, ensure_ascii=False)

'{"name": "get_weather_forecast", "description": "获取指定地点的天气预报", "parameters": {"type": "object", "properties": {"location": {"type": "string", "description": "要查询天气的地点"}, "date": {"type": "string", "description": "查询的日期"}}, "required": ["location", "date"]}}'